# <center>Analysis of Tests</center>

This file is an interactive notebook to analyze the data obtained from tests.

First the data must be read in.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(42)

# Read in the csv files
file_name = 'testOutput/testOutput_1590035296.csv'
data = pd.read_csv(file_name, header=0)

# Print the first 5 rows of the data
data.head()

,match_id,game_name,gdl_version,timestamp,startclock,sight_of,num_steps,role_1,player_1,player_1_score,role_2,player_2,player_2_score
0,randomhyper_0,montyhall,2,1590033080572,10,CANDIDATE,3,candidate,HYPER,0,random,RANDOM,100
1,randomhyper_1,montyhall,2,1590033082432,10,CANDIDATE,3,candidate,HYPER,100,random,RANDOM,100
2,randomhyper_2,montyhall,2,1590033084302,10,CANDIDATE,3,candidate,HYPER,0,random,RANDOM,100
3,randomhyper_3,montyhall,2,1590033086182,10,CANDIDATE,3,candidate,HYPER,100,random,RANDOM,100
4,randomhyper_4,montyhall,2,1590033088132,10,CANDIDATE,3,candidate,HYPER,100,random,RANDOM,100


Next, the average score of player 1 should be should be calculated.

In [2]:
total_score = data['player_1_score'].sum(skipna = True)
total_count = data['player_1_score'].count()

print("The win rate for the agent \"" + str(data['player_1'][0]) + "\" playing the game \"" + str(data['game_name'][0]) + "\" is " + str(total_score/total_count) + "% for " + str(total_count) + " games")

The win rate for the agent "HYPER" playing the game "montyhall" is 65.3% for 1000 games
